In [13]:
import pandas as pd     
import numpy as np 
import glob as glob


Base=[]
csv_files = glob.glob('Data/*.csv')

for filename in csv_files:
    if filename == "Data\\EAM_2020.csv":
        print("ok")
        data = pd.read_csv(filename,sep=",")
        
    else:
        data = pd.read_csv(filename,sep=";")
      
    Base.append(data)

ok


In [14]:
EAM08,EAM09,EAM10,EAM11,EAM12,EAM13,EAM14,EAM15,EAM16,EAM17,EAM18,EAM19,EAM20 = Base
Data_Merge= [EAM08,EAM09,EAM10,EAM11,EAM12,EAM13,EAM14,EAM15,EAM16,EAM17,EAM18,EAM19,EAM20]

#Assign each dataframe a variable 

In [15]:
EAM20.head()

,nordemp,nordest,dpto,ciiu4,periodo,c4r1c1n,c4r1c2n,c4r2c1e,c4r2c2e,c4r1c1,...,PERSOCU,PERSOESC,PPERYTEM,PERAPREN,SALAPREN,VALAGRI,VALORCOM,VALORCX,PORCVT,VALORVEN
0,141299,140327,19,2395,2020,0,0,0,0,0,...,20,20,20,1,9451.0,3179253,7579978,0,0,10408154
1,141302,140333,76,3290,2020,0,0,0,0,0,...,1,1,1,0,0.0,14965,103639,0,0,119213
2,141305,140336,5,2920,2020,0,0,0,0,0,...,5,5,5,0,0.0,74296,118900,0,0,276000
3,141308,140339,13,3290,2020,0,0,0,0,0,...,2,3,2,0,0.0,296942,201450,0,0,540842
4,141309,140340,8,3290,2020,0,0,0,0,0,...,80,80,80,1,2516.0,10741772,6599316,0,0,16374796


We star to studing of the datas

1. There are of null datas 
2. Correction of the difference forms of recolection and metodologys for each type of variable for each year
3. Interpretation of the variable for each type of the information that we get and data type 


In [16]:
#Data null, time of the wait:  2m 21.4s
missing=0
for i in Data_Merge:
        for col in i.columns:
            missing += i[col].isna().sum()
        for row in i.index:
            missing += i.loc[row,:].isna().sum()

print(missing)


822


In [17]:
#Cambiar el CIIU beacuse there differents labales in each dataframe

column_mapping = {"CIIU3": "CIIU", "ciiu3": "CIIU", "ciiu4": "CIIU", "ciiu_4": "CIIU", "CIIU4": "CIIU"}

for df in Data_Merge:
    df.rename(columns=column_mapping, inplace=True)

In [19]:
columna_ciiu_presente = True
for i, df in enumerate(Data_Merge):
    if "CIIU" not in df.columns:
        columna_ciiu_presente = False
        print("La columna CIIU no está presente en el DataFrame", i)

if columna_ciiu_presente:
    print("La columna CIIU está presente en todos los DataFrames.")

La columna CIIU3 está presente en todos los DataFrames.


In [22]:
conteo = EAM08["CIIU"].value_counts()





1810    757
3699    566
1551    440
2529    420
2220    285
       ... 
1600      3
2511      3
2430      3
3230      3
1542      3
Name: CIIU, Length: 126, dtype: int64
